In [1]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import random
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re

2024-12-20 09:47:03.732093: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 09:47:03.735818: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 09:47:03.744979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734688023.759392   17280 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734688023.763727   17280 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 09:47:03.787599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
import nltk
nltk.download('reuters')
nltk.download('punkt_tab')

[nltk_data] Downloading package reuters to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
def readData():
  data = ['This is a black dog', 'Pradeep"s cat', 'I love my cat', 'This is my name', 'Tharun loves his cat']
  dat = []
  for i in range(len(data)):
    for word in data[i].split():
      dat.append(word)
  return dat

def createBigram(data):
  listOfBigrams = []
  bigramCounts = {}
  unigramCounts = {}
  for i in range(len(data)-1):
    if i < len(data)-1 and data[i+1].islower():
        listOfBigrams.append((data[i], data[i + 1]))
        if (data[i], data[i+1]) in bigramCounts:
          bigramCounts[(data[i], data[i + 1])] += 1
        else:
          bigramCounts [(data[i], data[i + 1])] = 1
    if data[i] in unigramCounts:
      unigramCounts [data[i]] += 1
    else: unigramCounts [data[i]] = 1
    return listOfBigrams, unigramCounts, bigramCounts

def calcBigramProb(listOfBigrams, unigramCounts, bigramCounts):
  listOfProb = {}
  for bigram in listOfBigrams:
    word1 = bigram[0]
    word2 = bigram[1]
    print(bigram)
    listOfProb[bigram] = (bigramCounts.get(bigram))/(unigramCounts.get(word1))
    print(listOfProb)
  return listOfProb

data = readData()
listOfBigrams, unigramCounts, bigramCounts = createBigram(data)
print('Bigrams',listOfBigrams)
print('bigram count:',bigramCounts)
print('únigram:',unigramCounts)
bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)
print('bigram probability:',bigramProb)

Bigrams [('This', 'is')]
bigram count: {('This', 'is'): 1}
únigram: {'This': 1}
('This', 'is')
{('This', 'is'): 1.0}
bigram probability: {('This', 'is'): 1.0}


In [4]:
model = defaultdict(lambda: defaultdict(lambda: 0))

for sentence in reuters.sents():
  for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
    model[(w1, w2)][w3] += 1

for w1_w2 in model:
  total_count = float(sum (model [w1_w2].values()))
  for w3 in model [w1_w2]:
    model [w1_w2] [w3] /= total_count

dict(model["today","the"])

{'public': 0.05555555555555555,
 'European': 0.05555555555555555,
 'Bank': 0.05555555555555555,
 'price': 0.1111111111111111,
 'emirate': 0.05555555555555555,
 'overseas': 0.05555555555555555,
 'newspaper': 0.05555555555555555,
 'company': 0.16666666666666666,
 'Turkish': 0.05555555555555555,
 'increase': 0.05555555555555555,
 'options': 0.05555555555555555,
 'Higher': 0.05555555555555555,
 'pound': 0.05555555555555555,
 'Italian': 0.05555555555555555,
 'time': 0.05555555555555555}

In [5]:
text = ["today","the"]
sentence_finished = False
while not sentence_finished:
  r = random.random()
  accumulator = .0
  for word in model[tuple(text[-2:])].keys():
    accumulator += model[tuple(text[-2:])][word]
    if accumulator >= r:
      text.append(word)
      break
  if text[-2:] == [None, None]:
    sentence_finished = True
print(" ".join([t for t in text if t]))

today the options available .


In [6]:
data_text = """We all want the low-hanging fruit,
but let’s be honest – the low-hanging fruit is rotten, bruised, and covered with the grubby fingerprints of all the other spoiled brats pawing at it. There’s a time for easy wins, but easy only gets you so far. Sadly, I see too many SEOs putting days or weeks of effort into crafting the perfect low-value scheme, when that same time could’ve easily gone into content that has real staying power and drives sales. I’m obsessed with “Big Content” lately – resources that go beyond our narrow bins of blog posts, videos, and infographics. I’m going to show you how that obsession is paying off, and why building real content is easier than you think. If you want easy, then stop reading (this article is pretty long, and that sandwich won’t eat itself). The #1 attribute of big content is that it takes time and effort – it doesn’t have to be expensive, but you have to invest something into it (and, as they say, time is money). The problem with easy is that what’s easy for you is easy for everyone else, too. If anyone can do it, a tactic quickly loses impact. You can’t build a lasting competitive advantage with easy. Here’s what most people don’t get, though – once you get good at big, big gets easier. You learn how to be efficient, tap outside resources, and manage risks. The more you create big content, the more you see opportunities that weren’t there before. You have to put in the effort and make the mistakes – if you’re stuck on easy, big will always be out of your reach."""

def text_cleaner(text):
  newString = text.lower()
  newString = re.sub(r"'s\b"," ", newString)
  newString = re.sub("[^a-zA-Z]", " ", newString)
  long_words=[]
  for i in newString.split():
    if len(i)>=3:
      long_words.append(i)
  return (" ".join(long_words)).strip()

data_new = text_cleaner(data_text)

def create_seq(text):
  length = 30
  sequences = list()
  for i in range(length, len(text)):
    seq = text[i-length:1+1]
    sequences.append(seq)
  print('Total Sequences: %d' % len(sequences))
  return sequences #create sequences

sequences = create_seq(data_new)

Total Sequences: 1277
